<a href="https://www.kaggle.com/code/khunanonr/tuberculosis-chest-x-rays-shenzhen?scriptVersionId=131113948" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -Uqq fastbook

In [ ]:
import fastbook
# import the vision modules
from fastai.vision.all import *
from fastbook import *

In [ ]:
path = Path("../input/tuberculosis-chest-xrays-shenzhen/images/images")
path.ls()

In [ ]:
shenzen = pd.read_csv("../input/tuberculosis-chest-xrays-shenzhen/shenzhen_metadata.csv")

In [ ]:
shenzen['path'] = [(path/x) for x in shenzen['study_id']]

In [ ]:
#unique = shenzen['findings'].unique()
#map_lebel = {unique[i]:i for i in range(len(unique))}
#shenzen['label'] = shenzen['findings'].map(map_lebel)
shenzen['label'] = shenzen['study_id'].apply(lambda x: 0 if x[-5]=="0" else 1)

In [ ]:
shenzen.describe()

In [ ]:
shenzen.head()

# Data analysis

In [ ]:
sex = pd.DataFrame()
sex['normal'] = shenzen[shenzen['label'] == 0 ]['sex'].value_counts()
sex['sick'] = shenzen[shenzen['label'] == 1 ]['sex'].value_counts()
sex.plot.pie(subplots=True,figsize=(5, 5))

In [ ]:
shenzen.plot.box(y='age')

# Split Data

In [ ]:
def pandas_split(f, data=None) :
    index_train, index_val = f(data)
    return data.iloc[index_train], data.iloc[index_val]

In [ ]:
test_size = 0.1
f= TrainTestSplitter(test_size=test_size,random_state = 60, stratify=shenzen['label'])
train, test = pandas_split(f,shenzen)
train['label'].value_counts() test['label'].value_counts()

In [ ]:
train.head()

# Train

In [ ]:
def get_x(df) : return df['path']
def get_y(df) : return df['label']

tfms = [Rotate(max_deg=5, p=0.5),Zoom(draw=1.1, draw_x=0.5, draw_y=1, p=0.5), Flip(p=0.5)]
valid_size = 1/9
x_ray = DataBlock(blocks = (ImageBlock, CategoryBlock),
               get_x = get_x,
               get_y = get_y,
               splitter = RandomSplitter(valid_pct=valid_size, seed=42),
               item_tfms = Resize(224),
               batch_tfms = setup_aug_tfms(tfms)
                 )
dls = x_ray.dataloaders(train, bs=32)

In [ ]:
dls.valid.show_batch(max_n=10, nrows=4)

In [ ]:
dls.train.show_batch(max_n=20, nrows=4)

In [ ]:
# list_pre_train = [densenet121,densenet169, densenet201, densenet161]
# result = []
# for model in list_pre_train:
#  torch.cuda.empty_cache()
#  learn = cnn_learner(
#      dls,
#      model,
#      metrics=accuracy
#  )
#  learn.fine_tune(10, freeze_epochs=3)
#  result.append(learn.validate())

In [ ]:
learn = cnn_learner(
    dls,
    resnet50,
    metrics=accuracy
)

In [ ]:
# learn.fit_one_cycle(3,lr_steep)
# learn.unfreeze()
# lr_steep= learn.lr_find(suggest_funcs=(steep))
# lr_steep

In [ ]:
# base_lr = lr_steep[0]/2
# learn.fit_one_cycle(30, lr_max=slice(base_lr/100,base_lr), cbs=[ShowGraphCallback(), SaveModelCallback()])

In [ ]:
# learn.recorder.plot_loss()

In [ ]:
learn.fine_tune(30 , freeze_epochs=4, cbs=[ShowGraphCallback(), SaveModelCallback()])

In [ ]:
learn.validate()

In [ ]:
learn.show_results() 

## Nomal accuracy


In [ ]:
def nomal_accuracy(model, test):
    predn =  tensor([list(model.predict(x)[2]) for x in test['path']]);
    targn =  tensor(test['label'])
    return accuracy(predn, targn)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
confusion_m = interp.confusion_matrix()
interp.plot_confusion_matrix(figsize=(7,7))

In [ ]:
resnet101_model =  cnn_learner(
    dls,
    resnet101,
    metrics=accuracy
)
resnet101_model.load("resnet101")

In [ ]:
resnet101_model.validate()

In [ ]:
nomal_accuracy(learn, test)

In [ ]:
roc_accuracy(learn, test)

In [ ]:
dens =  cnn_learner(
    dls,
    densenet201,
    metrics=accuracy
)
dens.load("densnet201")

## Accuracy 
Accuracy = sensitivity x prevalence + specificity * (1 - prevalence)

In [ ]:
def confusion_matrix(pred, targ) :
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i in range(len(pred)):
        if not pred[i]:
            if targ[i] : fn+=1
            else: tn+=1
        else:
            if targ[i] :
                tp+=1
            else: fp+=1 
    return tp,fp,tn,fn

In [ ]:
def roc_accuracy(model, test) :
    pred =  tensor([list(learn.predict(x)[2]) for x in test['path']]);
    targ =  tensor(test['label'])
    pred, targ = flatten_check(pred.argmax(dim=-1), targ)
    tp, fp, tn, fn = confusion_matrix(pred, targ)
    sensitivity = tp/(tp+fn)
    specificity = tn/(fp+tn)
    prevalence = (pred == 1).sum()/len(pred)
    return sensitivity*prevalence + specificity * (1 - prevalence)